# Example 1: The Cratejoy Shipments API

This document illustrates how to interact with the Cratejoy shipments API. I'm using Python for illustration, but you can use whatever you want (cURL, Ruby, Java, JavaScript, etc). You don't want to use this from your Cratejoy theme, however, because you'd have to store your auth credentials in your publicly-accessible JavaScript files.

Here's an outline of this document:

- Import modules we'll need throughout the example
- Declare our auth credentials and the URL of the shipments API
- Use the endpoint to retrieve all of my shipments
- Mark a shipment as shipped

### Imports, auth, and the URL

In [12]:
import json
import requests


# Auth is your basic auth credentials ('username', 'password')
auth = ('blake', 'password')
base_url = 'http://api.cratejoy.com/v1/shipments/'

### Getting all of my shipments

First we'll use the python requests library to make a GET request to the shipments endpoint (and make sure the response came back normal).

In [13]:
response = requests.get(base_url, auth=auth)
print 'The Cratejoy API responded with status code:', response.status_code

response_json = response.json()

The Cratejoy API responded with status code: 200


#### Response data
Now let's look at the data. It's too much information to print out at all at once, but it boils down to the results of our query (along with some extra metadata). REST APIs often return data this way when you hit the index of some endpoint. In particular, we're interested in these fields:

- count: How many shipments I have in the system
- results: Complete data for the first 10 shipments
- next: The URL for next 10 shipments

In [20]:
print 'Fields:', response_json.keys()
print '\ncount:', response_json['count']
print '\nnext:', response_json['next']
print '\nresults:', response_json['results'][0:2]  # (just print the first 2)


Fields: [u'count', u'prev', u'results', u'next']

count: 18103

next: ?page=1

results: [{u'customer': {u'first_name': u'Rebecca', u'last_name': u'Plumlee', u'name': u'Rebecca Plumlee', u'num_orders': 1, u'country': u'US', u'total_revenue': 4497, u'store_settings': None, u'num_subscriptions': 1, u'id': 37722805, u'location': u'FL, US', u'subscription_status': u'active', u'type': u'customer', u'email': u'37722805@cjtest.com'}, u'status': u'unshipped', u'ship_address': {u'phone_number': u'', u'city': u'Altamonte Springs', u'country': u'US', u'company': u'', u'to': u'Rebecca Plumlee', u'state': u'FL', u'street': u'100 Spring Cove Trail', u'status': 1, u'type': u'address', u'zip_code': u'32714', u'status_message': None, u'id': 37722813, u'unit': u'', u'icon': u'United-States.png'}, u'fulfillments': [{u'status': u'open', u'total_cycles': 3, u'shipstation_dirty': True, u'fulfillment_date': 1437714000000, u'tax': 0, u'shipping': 299, u'shipment_id': 37722833, u'is_test': False, u'instance': {

#### ALL shipments
We only get 10 results at a time, but we can use the 'next' field to make queries in a loop until a response comes back with a 'next' field of None (at which point we know we have the last page). It's bad API ettiquette to do this too quickly, so we'll put a little lag time in the loop.

In [ ]:
import time

response_blobs = [response_json]  # this will be a list of all our of json blobs

while response_json['next']:
    
    response = requests.get(base_url + response_json['next'], auth=auth)
    response_json = response.json()
    
    response_blobs.append(response_json)
    
    time.sleep(.1)

### Marking as Shipped

We can mark individual shipments as "shipped" through the API. This works the same as in the merchant portal. If you have the notification active, the customer will automatically be notified.

Here we'll mark shipment #36014242 as shipped. First we need a JSON-encoded blob of data that represents the new status we're assigning to the shipment. Then we just make a PUT request to tell the Cratejoy back-end about the change. 

In [9]:
shipment_url = base_url + '36014242/'
mark_as_shipped_data = json.dumps({'status': 'shipped'})

response = requests.put(shipment_url, auth=auth, data=mark_as_shipped_data)
print 'The Cratejoy API responded with status code:', response.status_code

The Cratejoy API responded with status code: 401
